In [26]:
import matplotlib.pyplot as plt
%matplotlib inline
import nltk                            # Cleaning the data
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,SpatialDropout1D,Bidirectional
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [17]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices()) 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17994973869249935294
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8133044015714963998
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4839348633
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14802762287292926413
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15343173787345564870
physical_device_desc: "device: XLA_GPU device"
]


In [18]:
# nltk.download('punkt')

In [27]:
df = pd.read_pickle('Pickle_files/train_data.pkl')

In [45]:
hc =df[df['Target'] == 'Atheism']

In [46]:
hc

,Tweet,Target,Stance
100,dear lord thank u for all of ur blessings forg...,Atheism,AGAINST
101,blessed are the peacemakers for they shall be ...,Atheism,AGAINST
102,i am not conformed to this world i am transfor...,Atheism,AGAINST
103,salah should be prayed with focus and understa...,Atheism,AGAINST
104,and stay in your houses and do not display you...,Atheism,AGAINST
...,...,...,...
608,be amazeda real unicorns among us narwhals semst,Atheism,AGAINST
609,thjuly saw independence from the oppression o...,Atheism,AGAINST
610,thy kingdom come thy will be done on earth as ...,Atheism,AGAINST
611,aamirkhan says in pk tht fasting in the name o...,Atheism,AGAINST


In [58]:
train_X = hc['Tweet']
target = hc['Stance']

from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split( train_X, target , test_size = 0.2, random_state = 42)

In [59]:
# max length of the review

r_len=[]
for text in hc['Tweet']:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np

In [60]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 2140
max_words = 350
batch_size = 128
epochs = 6
num_classes=1

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [61]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)

In [62]:
print(X_train.shape,X_val.shape)

(410, 350) (103, 350)


In [63]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    return embedding_matrix
    

In [64]:
EMBEDDING_FILE = 'glove.6B/glove.6B.300d.txt'
embed_dim = 300 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.
(2140, 300)


In [65]:
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
Y_val = le.transform(Y_val)

Y_train = to_categorical(Y_train, dtype ="uint8") 
Y_val = to_categorical(Y_val, dtype ="uint8") 

In [66]:
Y_train

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [77]:
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model.add(SpatialDropout1D(0.25))
# model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Bidirectional(LSTM(64,return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 350, 300)          642000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 350, 300)          0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 128)               186880    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 829,267
Trainable params: 829,267
Non-trainable params: 0
_________________________________________________________________


In [81]:
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=15, batch_size=32, verbose=1)

Epoch 1/15
13/13 [==============================] - 1s 45ms/step - loss: 0.0144 - accuracy: 0.9951 - val_loss: 1.4836 - val_accuracy: 0.6893
Epoch 2/15
13/13 [==============================] - 1s 42ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 1.1841 - val_accuracy: 0.7184
Epoch 3/15
13/13 [==============================] - 1s 41ms/step - loss: 0.0137 - accuracy: 0.9927 - val_loss: 1.3643 - val_accuracy: 0.7184
Epoch 4/15
13/13 [==============================] - 1s 41ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 1.3433 - val_accuracy: 0.7087
Epoch 5/15
13/13 [==============================] - 1s 41ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 1.1810 - val_accuracy: 0.7476
Epoch 6/15
13/13 [==============================] - 1s 41ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 1.2546 - val_accuracy: 0.6990
Epoch 7/15
13/13 [==============================] - 1s 41ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.3033 - val_accuracy: 0.6990
Epoch 8/15
13

In [83]:
model.predict(X_val)

array([[9.9997020e-01, 2.4971110e-05, 4.8812208e-06],
       [3.1015006e-01, 3.1093019e-01, 3.7891978e-01],
       [1.9416224e-02, 6.9817441e-04, 9.7988564e-01],
       [5.6589567e-03, 1.6662093e-01, 8.2772011e-01],
       [1.0687821e-03, 4.1838828e-03, 9.9474734e-01],
       [9.9989188e-01, 4.5857993e-05, 6.2173924e-05],
       [6.7865808e-04, 5.7751691e-04, 9.9874383e-01],
       [9.9996054e-01, 3.1193205e-05, 8.1923463e-06],
       [3.7846013e-04, 1.3458034e-03, 9.9827576e-01],
       [8.2294595e-05, 8.3064544e-05, 9.9983466e-01],
       [9.9986970e-01, 1.2079585e-04, 9.5036212e-06],
       [9.2109942e-01, 1.4447543e-03, 7.7455811e-02],
       [9.9995852e-01, 3.8726346e-05, 2.7573240e-06],
       [2.3468649e-01, 2.5335771e-01, 5.1195580e-01],
       [9.9949944e-01, 1.8992396e-04, 3.1064541e-04],
       [5.3653662e-04, 5.1028505e-02, 9.4843501e-01],
       [9.9416310e-01, 2.6036603e-03, 3.2332586e-03],
       [9.9996078e-01, 3.6819594e-05, 2.3497048e-06],
       [3.6844797e-03, 1.122